# Cutucando os dados com Gremlin em Python

Primeiro importamos todas as paradas que vamos precisar daqui pra frente.

In [1]:
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order

O nosso banco de dados precisa estar rodando em outro processo. Pode ser no nosso computador ou remoto, mas tem que estar em algum lugar. Se garantindo que está rodando local, vamos nos conectar a partir do código Python.

In [2]:
g = Graph().traversal().withRemote(
    DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

Nesse momento a gente já sabe que tá de boas. Temos uma variável que é uma instância de uma origem de transversais da linguagem Gremlin. Com essa bixinha "g" qualquer transversal de consulta pode ser construída. Agora é que a brincadeira começa.

Primeiro precisamos criar uns dados pra consultar. Vamos adicionar dados ao banco.

In [3]:
g.addV("person").property("name", "Betina").as_("betina"). \
    addV("person").property("name", "Fernanda").as_("fernanda"). \
    addV("person").property("name", "Gustavo").as_("gustavo"). \
    addV("person").property("name", "Daniel").as_("daniel"). \
    addV("person").property("name", "Joana").as_("joana"). \
    addV("person").property("name", "Marcos").as_("marcos"). \
    addV("attraction").property("title", "Galo").property("location", "Recife").as_("galo"). \
    addV("attraction").property("title", "Recife Antigo").property("location", "Recife").as_("antigo"). \
    addV("attraction").property("title", "A Vaidosa").property("location", "Olinda").as_("vaidosa"). \
    addV("attraction").property("title", "Bloco Acorda").property("location", "Bezerros").as_("bezerros"). \
    addE("goes_to").from_("betina").to("galo"). \
    addE("goes_to").from_("betina").to("antigo"). \
    addE("goes_to").from_("marcos").to("galo"). \
    addE("goes_to").from_("marcos").to("bezerros"). \
    addE("goes_to").from_("fernanda").to("galo"). \
    addE("goes_to").from_("fernanda").to("vaidosa"). \
    addE("goes_to").from_("gustavo").to("galo"). \
    addE("goes_to").from_("daniel").to("galo"). \
    addE("goes_to").from_("joana").to("bezerros"). \
    addE("goes_to").from_("gustavo").to("bezerros"). \
    addE("knows").from_("joana").to("betina"). \
    addE("knows").from_("betina").to("marcos"). \
    addE("knows").from_("betina").to("fernanda"). \
    addE("knows").from_("gustavo").to("joana"). \
    addE("knows").from_("marcos").to("gustavo"). \
    addE("knows").from_("daniel").to("marcos"). \
    addE("knows").from_("daniel").to("betina"). \
    iterate()

[['addV', 'person'], ['property', 'name', 'Betina'], ['as', 'betina'], ['addV', 'person'], ['property', 'name', 'Fernanda'], ['as', 'fernanda'], ['addV', 'person'], ['property', 'name', 'Gustavo'], ['as', 'gustavo'], ['addV', 'person'], ['property', 'name', 'Daniel'], ['as', 'daniel'], ['addV', 'person'], ['property', 'name', 'Joana'], ['as', 'joana'], ['addV', 'person'], ['property', 'name', 'Marcos'], ['as', 'marcos'], ['addV', 'attraction'], ['property', 'title', 'Galo'], ['property', 'location', 'Recife'], ['as', 'galo'], ['addV', 'attraction'], ['property', 'title', 'Recife Antigo'], ['property', 'location', 'Recife'], ['as', 'antigo'], ['addV', 'attraction'], ['property', 'title', 'A Vaidosa'], ['property', 'location', 'Olinda'], ['as', 'vaidosa'], ['addV', 'attraction'], ['property', 'title', 'Bloco Acorda'], ['property', 'location', 'Bezerros'], ['as', 'bezerros'], ['addE', 'goes_to'], ['from', 'betina'], ['to', 'galo'], ['addE', 'goes_to'], ['from', 'betina'], ['to', 'antigo']

In [4]:
g.V().hasLabel("person").toList()

[v[8384],
 v[40964176],
 v[40964344],
 v[4256],
 v[8320],
 v[40968440],
 v[4304],
 v[4272],
 v[4264],
 v[4216],
 v[40964168],
 v[4328],
 v[40964216],
 v[4224],
 v[4288],
 v[40964328],
 v[40964256],
 v[40972536]]

Como consultamos um vértice usando uma propriedade que a gente já sabe?

In [6]:
g.V().has("name", "Betina").valueMap().next()

{'name': ['Betina']}

Quem são os amigos de Betina?

In [9]:
g.V().has("name", "Betina").out("knows").values("name").toList()

['Marcos', 'Marcos', 'Marcos', 'Fernanda', 'Fernanda', 'Fernanda']

Quantos vértices temos no banco?

In [7]:
g.V().count().next()

10

Quantas arestas temos no banco?

In [8]:
g.E().count().next()

17

Quais são as festas que temos no banco?

In [9]:
g.V().hasLabel("attraction").values("title").toList()

['Bloco Acorda', 'Galo', 'Recife Antigo', 'A Vaidosa']

Quantas pessoas vão pra cada uma das festas?

In [10]:
g.V().hasLabel("attraction").as_("bloco", "total_people"). \
    select("bloco", "total_people"). \
        by("title"). \
        by(__.inE("goes_to").count()). \
    order().by(__.select('total_people'), Order.decr).toList()

[{'bloco': 'Galo', 'total_people': 5},
 {'bloco': 'Bloco Acorda', 'total_people': 3},
 {'bloco': 'Recife Antigo', 'total_people': 1},
 {'bloco': 'A Vaidosa', 'total_people': 1}]

Quem se conhece e tá indo pra mesma festa?

In [11]:
g.V().match(
    __.as_("a").out("knows").as_("b"), \
    __.as_("a").out("goes_to").as_("event"), \
    __.as_("b").out("goes_to").as_("event"), \
).select("a", "b", "event").by("name").by("name").by("title").toList()

[{'a': 'Marcos', 'b': 'Gustavo', 'event': 'Bloco Acorda'},
 {'a': 'Marcos', 'b': 'Gustavo', 'event': 'Galo'},
 {'a': 'Gustavo', 'b': 'Joana', 'event': 'Bloco Acorda'},
 {'a': 'Daniel', 'b': 'Betina', 'event': 'Galo'},
 {'a': 'Daniel', 'b': 'Marcos', 'event': 'Galo'},
 {'a': 'Betina', 'b': 'Fernanda', 'event': 'Galo'},
 {'a': 'Betina', 'b': 'Marcos', 'event': 'Galo'}]